メモ
I91,95,84あたりは多い

<table>
  <thead>
  <tr>
    <td>更新日時</td>
    <td>変更内容</td>    
  </tr>
  </thead>
  <tr>
    <td>0611 1230</td>
    <td>locationあり、officerなし。良さげ。</td> 
  </tr>
  <tr>
    <td>0613 1000</td>
    <td>dayを追加。location/officerは追加</td> 
  </tr>
  <tr>
    <td>0613 1430</td>
    <td>dayを追加。location/officerは追加</td> 
  </tr>
  <tr>
    <td>0613 1730</td>
    <td>locationのうち小さいものを除去するなど。officerなし。良さげ。</td> 
  </tr>
  <tr>
    <td>0613 1800</td>
    <td>location(>50)、officer(>1000)を追加。安定して0.88でる。</td> 
  </tr>
</table>

In [96]:
import pandas as pd
import numpy as np
from datetime import datetime

In [97]:
df = pd.read_csv("test.csv")
ctlist = pd.read_csv("CTlist.csv")

In [98]:
df.head(3)

,state,stop_date,stop_time,location_raw,county_name,county_fips,fine_grained_location,police_department,driver_gender,driver_age_raw,...,driver_race_raw,driver_race,violation_raw,violation,search_conducted,search_type_raw,search_type,contraband_found,officer_id,stop_duration
0,CT,2014-10-13,07:18,orange,New Haven County,9009.0,"00000 S RT 15 (ORANGE, T107)",State Police,M,42,...,Black,Black,Moving Violation,Moving violation,False,NaN,NaN,False,19172,1-15 min
1,CT,2014-11-25,03:25,woodbury,Litchfield County,9005.0,RT 6,State Police,F,59,...,White,White,Registration,Registration/plates,False,NaN,NaN,False,1000002390,1-15 min
2,CT,2015-03-23,09:32,norwich,New London County,9011.0,i395nb x82,State Police,M,42,...,White,White,Other/Error,Other,False,NaN,NaN,False,1000003180,30+ min


In [99]:
ctlist.head(3)

,Unnamed: 0,Town,Designation,Date,Land area,Population,Form of,County,Council of Governments,Native American,X,Y
0,0,andover,Town,1848,15.46,"3,303",Town meeting,Tolland County,Capitol Region,NaN,41.737321,-72.370360
1,1,ansonia,City,1889,6.03,"19,249",Mayor-council,New Haven County,Naugatuck Valley,NaN,41.346208,-73.078997
2,2,ashford,Town,1714,38.79,"4,100",Town meeting,Windham County,Northeast CT,NaN,41.873066,-72.121616


以下、データの成形を行なっていく。

Violationに関しては、カンマ区切りにされているviolationの種類をstr.splitで分割した上で、全てのviolationの種類をまとめたリストを作成する。このリストをカラム名とし、各indexについて対応するviolationの値を1とする。

In [100]:
# violationの種類を検索
df_violation = df["violation"].str.split(',',expand=True)
violation_list = np.array([])

"""
for i in range(0,4):
    violation_list = np.append(violation_list,df_violation[0].unique())
violation_list = np.unique(violation_list)
"""

violation_list = np.array(['Cell phone', 'Equipment', 'License', 'Lights', 'Moving violation',
       'Other', 'Registration/plates', 'Safe movement', 'Seat belt',
       'Speeding', 'Stop sign/light'], dtype=object)

for list in violation_list:
    df["violation_"+list] = np.zeros(df.contraband_found.count())
    for i in range(0,4):
        df.loc[df_violation[i]==list,"violation_"+list]=1

In [101]:
# stop_durationを数値に直す
# 2回目以降の実行ではエラー
df.loc[df["stop_duration"]=="1-15 min","stop_duration"]=0
df.loc[df["stop_duration"]=="16-30 min","stop_duration"]=1
df.loc[df["stop_duration"]=="30+ min","stop_duration"]=2

# driver_genderを数値に直す
# 2回目以降の実行ではエラー
df.loc[df["driver_gender"]=="M","driver_gender"]=0
df.loc[df["driver_gender"]=="F","driver_gender"]=1

In [102]:
df['stop_time'] = pd.to_datetime(df['stop_time'])
df['stop_date'] = pd.to_datetime(df['stop_date'])
df["stop_weekday_name"] = df["stop_date"].dt.weekday_name
df["stop_month"] = df["stop_date"].dt.month    
df["stop_year"] = df["stop_date"].dt.year
df["stop_hour"] = df["stop_time"].dt.hour
df["stop_day"] = df["stop_date"].dt.day

In [103]:
df["location_raw"] = df["location_raw"].fillna("N/A")
df["fine_grained_location"] = df["fine_grained_location"].fillna("N/A")

In [104]:
df["location_exit"] = 0
df.loc[df["fine_grained_location"].str.contains("X"),"location_exit"] = 1
df.loc[df["fine_grained_location"].str.contains("x"),"location_exit"] = 1
df.loc[df["fine_grained_location"].str.contains("sex"),"location_exit"] = 0

df["location_route"] = 0
df.loc[df["fine_grained_location"].str.contains("rt"),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("rte "),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("RT."),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("RT "),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("rT "),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("Route"),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("Rte"),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("Rt"),"location_route"] = 1

df["location_00000"] = 0
df.loc[df["fine_grained_location"].str.contains("000"),"location_00000"] = 1

In [105]:
df.loc[df["location_raw"]=="e lyme","location_raw"] = "east lyme"
df.loc[df["location_raw"]=="e hartford","location_raw"] = "east hartford"
df.loc[df["location_raw"]=="e granby","location_raw"] = "east granby"
df.loc[df["location_raw"]=="no stonington","location_raw"] = "north stonington"
df.loc[df["location_raw"]=="no canaan","location_raw"] = "north canaan"
df.loc[df["location_raw"]=="no branford","location_raw"] = "north branford"
df.loc[df["location_raw"]=="falls village","location_raw"] = "canaan" #CDP
df.loc[df["location_raw"]=="ivoryton","location_raw"] = "essex" #CDP
df.loc[df["location_raw"]=="sprague - baltic","location_raw"] = "sprague" #CDP 
df.loc[df["location_raw"]=="sprague - versailles","location_raw"] = "sprague" #? #
df.loc[df["location_raw"]=="storrs","location_raw"] = "mansfield" # CDP
df.loc[df["location_raw"]=="centerbrook","location_raw"] = "essex" #hamletって場所らしい
df.loc[df["location_raw"]=="griwold","location_raw"] = "griswold"
df.loc[df["location_raw"]=="uncasville","location_raw"] = "montville"
df.loc[df["location_raw"]=="winsted","location_raw"] = "litchfield"
df.loc[df["location_raw"]=="washighton","location_raw"] = "washington"
df.loc[df["location_raw"]=="danielson","location_raw"] = "killingly"
df.loc[df["location_raw"]=="t165","location_raw"] = "windsor locks"

df.loc[(df["county_fips"].isnull()) & (df["fine_grained_location"].str.contains("T045")),["location_raw","county_name","county_fips"]] = ["east lyme","New London County","9011.0"]
df.loc[(df["county_fips"].isnull()) & (df["fine_grained_location"].str.contains("t045")),["location_raw","county_name","county_fips"]] = ["east lyme","New London County","9011.0"]

In [106]:
df = pd.get_dummies(df,columns=["stop_month"], prefix='stop_month', prefix_sep='_')
df = pd.get_dummies(df,columns=["stop_year"], prefix='stop_year', prefix_sep='_')
df = pd.get_dummies(df,columns=["stop_weekday_name"], prefix='stop_day', prefix_sep='_')
df = pd.get_dummies(df,columns=["stop_hour"], prefix='stop_hour', prefix_sep='_')

df = pd.get_dummies(df,columns=["driver_race"],prefix='race',prefix_sep='_')

df = pd.get_dummies(df,columns=["search_type"], prefix='search', prefix_sep='_')

df = pd.get_dummies(df,columns=["county_name"], drop_first=True, prefix='county', prefix_sep='_')

In [107]:
df = df.drop(["stop_date","stop_time","search_type_raw","search_conducted","county_fips"
              ,"driver_race_raw","violation","violation_raw","state"
              ,"police_department","driver_age"],axis=1)

In [108]:
df["contraband_found"] = df["contraband_found"]*1

In [109]:
ctlist["Population"] = ctlist["Population"].str.replace(',','').astype(np.int)

In [110]:
df.loc[df["location_raw"].isnull(),"location_raw"]="N/A"

In [111]:
population_dict = dict(zip(ctlist['Town'], ctlist['Population']))
area_dict = dict(zip(ctlist['Town'], ctlist['Land area']))
X_dict = dict(zip(ctlist['Town'], ctlist['X']))
Y_dict = dict(zip(ctlist['Town'], ctlist['Y']))
population_dict.update({"N/A":ctlist["Population"].mean()})
area_dict.update({"N/A":ctlist["Land area"].mean()})
X_dict.update({"N/A":ctlist["X"].mean()})
Y_dict.update({"N/A":ctlist["Y"].mean()})

In [112]:
df["population"] = df["location_raw"].replace(population_dict).astype(np.int)
df["area"] = df["location_raw"].replace(area_dict).astype(np.float)
df["X"] = df["location_raw"].replace(X_dict).astype(np.float)
df["Y"] = df["location_raw"].replace(Y_dict).astype(np.float)

In [113]:
df["location_raw"].unique()
location_list = np.array(['columbia', 'sherman', 'colchester', 'west haven', 'haddam',
       'mansfield', 'stonington', 'marlborough', 'griswold', 'milford',
       'norwalk', 'bridgewater', 'tolland', 'deep river', 'union',
       'roxbury', 'cornwall', 'woodbridge', 'shelton', 'brooklyn',
       'danbury', 'wallingford', 'groton', 'litchfield', 'oxford',
       'naugatuck', 'new london', 'brookfield', 'plainfield', 'scotland',
       'wethersfield', 'north haven', 'middletown', 'north canaan',
       'trumbull', 'bethel', 'westbrook', 'killingly', 'lisbon',
       'bridgeport', 'somers', 'old saybrook', 'east granby', 'vernon',
       'winchester', 'burlington', 'east hartford', 'ellington', 'kent',
       'new milford', 'hartford', 'morris', 'willington', 'middlebury',
       'north stonington', 'chester', 'middlefield', 'stafford',
       'westport', 'old lyme', 'thompson', 'goshen', 'manchester',
       'windsor', 'stamford', 'farmington', 'essex', 'east haddam',
       'stratford', 'colebrook', 'waterbury', 'greenwich', 'harwinton',
       'guilford', 'beacon falls', 'madison', 'watertown', 'waterford',
       'lebanon', 'windsor locks', 'new canaan', 'franklin', 'salisbury',
       'fairfield', 'preston', 'new haven', 'putnam', 'andover',
       'woodbury', 'salem', 'ashford', 'enfield', 'clinton', 'cromwell',
       'bolton', 'southbury', 'newtown', 'durham', 'ansonia', 'montville',
       'washington', 'branford', 'east lyme', 'norwich', 'new hartford',
       'thomaston', 'hamden', 'canaan', 'darien', 'hebron', 'pomfret',
       'voluntown', 'derby', 'ledyard', 'new fairfield', 'eastford',
       'new britain', 'meriden', 'west hartford', 'bristol', 'windham',
       'berlin', 'rocky hill', 'monroe', 'glastonbury', 'killingworth',
       'orange', 'torrington', 'east windsor', 'southington', 'bozrah',
       'seymour', 'bethany', 'barkhamsted', 'cheshire', 'east haven',
       'norfolk', 'east hampton', 'sharon', 'sterling', 'north branford',
       'plainville', 'hampton', 'avon', 'sprague', 'woodstock', 'chaplin',
       'coventry', 'prospect', 'south windsor', 'canton', 'canterbury',
       'newington', 'bethlehem', 'portland', 'hartland', 'warren', 'lyme',
       'easton', 'plymouth', 'bloomfield', 'ridgefield', 'suffield',
       'wolcott', 'simsbury', 'granby','wilton','orange'],
      dtype=object)

In [82]:
df["officer_id"].unique()[df["officer_id"].value_counts()>50]
officer_list = np.array(['55887736', '770626977', '1000002961', '1000002376', '70583331',
       '1000002714', '1000003026', '1000002219', '298463910',
       '1000002220', '1000003191'],
      dtype=object)

In [114]:
%%time
for list in location_list:
    df["location_"+list] = np.zeros(df.contraband_found.count())
    df.loc[df["location_raw"]==list,"location_"+list]=1

CPU times: user 5.56 s, sys: 5.43 s, total: 11 s
Wall time: 11 s


In [84]:
%%time
for list in officer_list:
    df["officer_"+str(list)] = np.zeros(df.contraband_found.count())
    df.loc[df["location_raw"]==str(list),"officer_"+str(list)]=1

CPU times: user 13.6 s, sys: 1.74 s, total: 15.4 s
Wall time: 1.93 s


In [115]:
df["location_I84"] = 0
df.loc[(df["fine_grained_location"].str.contains("84")),"location_I84"] = 1

df["location_I91"] = 0
df.loc[(df["fine_grained_location"].str.contains("91")) & ~(df["fine_grained_location"].str.contains("T091")),"location_I91"] = 1

df["location_I95"] = 0
df.loc[(df["fine_grained_location"].str.contains("95")) & ~(df["fine_grained_location"].str.contains("T095")),"location_I95"] = 1

df["location_I395"] = 0
df.loc[(df["fine_grained_location"].str.contains("395")) & ~(df["fine_grained_location"].str.contains("T395")),"location_I395"] = 1

In [116]:
df = df.drop(["fine_grained_location","location_raw","officer_id"],axis=1)

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93994 entries, 0 to 93993
Columns: 255 entries, driver_gender to location_I395
dtypes: float64(181), int64(13), uint8(61)
memory usage: 144.6 MB


In [118]:
df.to_csv('test_clean_06150000.csv',sep=',',index=False)